====================================================================

### 1. 프로젝트 
- 프로젝트명 : Blitz 수요 예측 모델 및 경로 최적화
- 기간 : 2024년 08월 16일 ~ 2024년 11월 01일
- 모형 : 경로 최적화 모델(Route Optimization)

### 2. 작성자
- 회사명 : 에이젠 글로벌
- 이름  : 남건우
- 직급  : 매니저

### 3. 데이터
- size(전처리 후): 6537 건
        
====================================================================

# 1. 데이터 전처리

In [1]:
# 0. 함수 호출
from def_model import *

# 1. Base
import copy
import time
from datetime import timedelta
from datetime import datetime
import itertools

from random import *
from tqdm import tqdm

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings(action='ignore')

In [2]:
data_dir = '../../../Blitz/TTA/data/pre/blitz_data_route.pickle'
df = pd.read_pickle("blitz_data_route.pickle")

In [3]:
# 2024년 8월 필터
df = df[(df['pickup_done_at_(1st_attempt)'].dt.year == 2024) & (df['pickup_done_at_(1st_attempt)'].dt.month == 8)]

# 허브 3곳 필터
blitz_hub_list = ['Blitz Station - Jakarta Timur', 'Blitz Station - Jakarta Barat', 'Blitz Head Office']
df = df[df.blitz_hub.isin(blitz_hub_list)]

# midmile_arrived_time null값 필터
df = df[df.midmile_arrived_at != '-']

# 기존 경로 추적 시 필요한 driver name 필터
df = df.dropna(subset=['driver_name', 'pickup_done_at_(1st_attempt)'])

In [4]:
# 주문 묶음을 나누는 기준인 time block 설정
bins = [i for i in range(0, 27, 3)]
labels = ['{0:02d}-{1:02d}'.format(i, i+3) for i in range(0, 24, 3)]

df['midmile_arrived_at'] = pd.to_datetime(df['midmile_arrived_at'], errors = 'coerce')
df['time_block'] = pd.cut(df['midmile_arrived_at'].dt.hour, bins=bins, labels=labels, right=False)
df['time_block'] = df['midmile_arrived_at'].dt.date.astype(str) + ' ' + df['time_block'].astype(str)

In [5]:
# 주문 마감기한 계산
df['due_time'] = df.apply(calculate_duetime, axis=1)

In [6]:
# 인덱스 초기화
df.reset_index(inplace=True)

In [7]:
df.shape

(6537, 49)

# 2. 최종 모형

In [8]:
distance_matrix, duration_matrix, coords = euclidean_distance_duration_matrix(df)

In [9]:
due_time = df.due_time.tolist()
demand = df.weight.tolist()
capacity = 180

In [10]:
dates = pd.date_range(start='2024-08-01', end='2024-08-31', freq='D')
time_blocks = ['{0:02d}-{1:02d}'.format(i, i+3) for i in range(0, 24, 3)]
blitz_hub_list = ['Blitz Station - Jakarta Timur', 'Blitz Station - Jakarta Barat', 'Blitz Head Office']

In [11]:
solution_list = []
code_start_time = time.time()

for date, time_block in tqdm(itertools.product(dates, time_blocks), total=len(dates) * len(time_blocks)):
        date_str = date.strftime('%Y-%m-%d') + ' ' + time_block
        for blitz_hub in blitz_hub_list:

            start_time = time.time()
            tour_solution = VNS(df, date_str, blitz_hub, distance_matrix, duration_matrix, due_time, demand, capacity, coords)
            end_time = time.time()

            if len(tour_solution) > 0:
                solution_dict = {
                    "time_block_and_hub": date_str + " " + blitz_hub,
                    "time_block" : date_str,
                    "blitz_hub": blitz_hub,
                    "tour_solution": tour_solution,
                    "total_distance": total_distance(tour_solution, distance_matrix),
                    "duration": end_time - start_time,
                }
                solution_list.append(solution_dict)

code_end_time = time.time()
print(f"\nTotal duration: {code_end_time - code_start_time:.2f} seconds.")

100%|██████████| 248/248 [1:29:34<00:00, 21.67s/it]


Total duration: 5374.68 seconds.


In [12]:
solution_df = pd.DataFrame(solution_list)
total_tour_solution = [item for sublist in solution_df.tour_solution.tolist() for item in sublist] 

In [13]:
improved_distance = solution_df.total_distance.sum()

In [14]:
original_tour = [
    [blitz_hub_to_i(path_df.blitz_hub.iloc[0])] + path_df.index.tolist()
    for path_id, path_df in df.groupby(['driver_name', 'pickup_done_at_(1st_attempt)']) if not path_df.empty
]
original_distance = total_distance(original_tour, distance_matrix)

In [15]:
print(f'절감률은 {(original_distance - improved_distance) / original_distance * 100:.2f}% 입니다')

절감률은 29.60% 입니다
